In [3]:
import numpy as np
import pandas as pd

In [4]:
def f1(x, y):
    return x * y

def f2(x, y):
    return np.exp(x + y)

In [5]:
def numerical_integral(f, x_range, y_range, dx, dy):
    x_start, x_end = x_range
    y_start, y_end = y_range

    # dx = (x_end - x_start) / (n + 1) -> n + 1 = 總寬度 / dx
    nx = int(round((x_end - x_start) / dx))
    ny = int(round((y_end - y_start) / dy))
    
    total_integral = 0
    
    # [xi, xi+1] x [yj, yj+1]
    for i in range(nx):
        for j in range(ny):
            xi = x_start + i * dx
            xi1 = xi + dx
            yj = y_start + j * dy
            yj1 = yj + dy

            # Corners
            corners = f(xi, yj) + f(xi, yj1) + f(xi1, yj) + f(xi1, yj1)
            # Edge midpoints
            mid_x = (xi + xi1) / 2
            mid_y = (yj + yj1) / 2
            edges = 2 * (f(mid_x, yj) + f(mid_x, yj1) + f(xi, mid_y) + f(xi1, mid_y))
            # Center point
            center = 4 * f(mid_x, mid_y)
            cell_sum = (corners + edges + center)
            total_integral += cell_sum
    # dx * dy / 16
    return total_integral * (dx * dy / 16)

In [6]:
# Exact Solutions
# f1: integral of x*y from 0 to 1 and 0 to 3 is 2.25
exact_f1 = 2.25
# f2: integral of e^(x+y) is (e^1 - 1)*(e^3 - 1)
exact_f2 = (np.exp(1) - 1) * (np.exp(3) - 1)

# delta_x, delta_y

test_pairs = [
    (1.0, 3.0),    
    (0.5, 1.5),    
    (0.25, 0.75),  
    (0.1, 0.3)     
]

In [7]:

results = []
for dx, dy in test_pairs:

    res_f1 = numerical_integral(f1, (0, 1), (0, 3), dx, dy)
    err_f1 = abs(res_f1 - exact_f1)
    
    res_f2 = numerical_integral(f2, (0, 1), (0, 3), dx, dy)
    err_f2 = abs(res_f2 - exact_f2)
    
    results.append({
        'Delta_x': dx,
        'Delta_y': dy,
        'f1_Result': res_f1,
        'f1_Error': err_f1,
        'f2_Result': res_f2,
        'f2_Error': err_f2
    })

In [8]:
df = pd.DataFrame(results)
print("--- Exact answer ---")
print(df.to_string(index=False))
print(f"\nf1 Exact answe: {exact_f1}")
print(f"f2 Exact answe: {exact_f2:.6f}")

--- Exact answer ---
 Delta_x  Delta_y  f1_Result  f1_Error  f2_Result  f2_Error
    1.00     3.00       2.25       0.0  39.527795  6.733464
    0.50     1.50       2.25       0.0  34.495895  1.701563
    0.25     0.75       2.25       0.0  33.220931  0.426600
    0.10     0.30       2.25       0.0  32.862642  0.068311

f1 Exact answe: 2.25
f2 Exact answe: 32.794331


In [9]:
# Exact answer
true_val_f1 = 2.25
true_val_f2 = (np.exp(1) - 1) * (np.exp(3) - 1)

def numerical_integration_hint(func, x_limit, y_limit, n_intervals, m_intervals):

    x0, xn = x_limit
    y0, ym = y_limit
    
    dx = (xn - x0) / n_intervals
    dy = (ym - y0) / m_intervals
    
    total_sum = 0.0

    for i in range(n_intervals):
        for j in range(m_intervals):
            xi = x0 + i * dx
            xi_next = x0 + (i + 1) * dx
            yj = y0 + j * dy
            yj_next = y0 + (j + 1) * dy
        
            xi_mid = (xi + xi_next) / 2
            yj_mid = (yj + yj_next) / 2
            
            term1 = func(xi, yj) + func(xi, yj_next) + func(xi_next, yj) + func(xi_next, yj_next)
            
            term2 = 2 * (func(xi_mid, yj) + func(xi_mid, yj_next) + 
                         func(xi, yj_mid) + func(xi_next, yj_mid))
            
            term3 = 4 * func(xi_mid, yj_mid)
            
            total_sum += (term1 + term2 + term3)
            
    result = (dx * dy / 16.0) * total_sum
    return result

test_cases = [
    (1, 1),    
    (2, 2),    
    (4, 4),    
    (10, 10)   
]

print(f"{'Grid (nx, ny)':<15} | {'f1 Result':<12} | {'f1 Error':<12} | {'f2 Result':<12} | {'f2 Error':<12}")
print("-" * 75)

for nx, ny in test_cases:
    res1 = numerical_integration_hint(f1, (0, 1), (0, 3), nx, ny)
    err1 = abs(res1 - true_val_f1)
    
    res2 = numerical_integration_hint(f2, (0, 1), (0, 3), nx, ny)
    err2 = abs(res2 - true_val_f2)
    
    print(f"({nx}, {ny}):{' ': <6} | {res1:.6f}     | {err1:.2e}     | {res2:.6f}     | {err2:.6e}")

Grid (nx, ny)   | f1 Result    | f1 Error     | f2 Result    | f2 Error    
---------------------------------------------------------------------------
(1, 1):       | 2.250000     | 0.00e+00     | 39.527795     | 6.733464e+00
(2, 2):       | 2.250000     | 0.00e+00     | 34.495895     | 1.701563e+00
(4, 4):       | 2.250000     | 0.00e+00     | 33.220931     | 4.265998e-01
(10, 10):       | 2.250000     | 0.00e+00     | 32.862642     | 6.831100e-02
